In [1]:
!export LC_ALL=en_US.UTF-8
from embedding import *

In [2]:
import json
from pprint import pprint
import re
import numpy as np
import sys
import math
import random

import torch
try:
    import nltk
except:
    !pip install nltk
    import nltk
    nltk.download('punkt')
    nltk.download('stopwords')
np.random.seed(0)

In [3]:
data = json.load(open('../train-v1.1.json'))

In [4]:
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltkStopWords = stopwords.words('english')
punctuations = [',', '?', '.', '-',]

[nltk_data] Downloading package stopwords to /home/ra2630/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ra2630/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
def extractor(data):
    contexts = []
    qas = []
    for i in range(len(data["data"])):
        for j in range(len(data["data"][i]["paragraphs"])):
            contexts.append(data["data"][i]["paragraphs"][j]["context"])
            qas.append(data["data"][i]["paragraphs"][j]["qas"])
    return (contexts,qas)

In [6]:
CapPassage = True

from nltk.tokenize import word_tokenize
contexts,qas = extractor(data)

def find_sub_list(sl,l):
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind,ind+sll
    return (-1,-1)

def capPassage(passage,left, right,cap_length = 20):
    y = np.zeros(cap_length)
    left = max(0,left - int((cap_length - len(answer))/2))
    right = min(right + int((cap_length + len(answer))/2), cap_length)
    if(left < 0):
        left = 0
    if(right > len(passage)):
        right = len(passage)
    return passage[left:right]
    
def findAnsVec(answer,passage):
    ans = np.zeros((len(passage)))
    start,end = find_sub_list(answer,passage)
    if(start==-1):
        start = passage.index(answer[0])
        end = start + len(answer)
    ans[start:end] = 1
    return ans

In [7]:
n_words = 10
X_train_comp_all = []
X_train_comp_with_answer_marked_all = []
X_train_ans_all = []
X_train_comp_answer_label_all = []
X_train_sentence_label_all = []
Y_train_ques_all = []
invalid = 0
for i,context in enumerate(contexts):
    passage = word_tokenize(context.lower())

    for j,_ in enumerate(qas[i]):
        try:
            question = word_tokenize(qas[i][j]['question'].lower())
            answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
            start,end = find_sub_list(answer,passage)
            if start == -1:
                invalid = invalid+1
                continue
            marked_comp = np.zeros(len(passage))
            marked_comp[start:end] = 1
            left = max(0,start - n_words)
            right = min(len(passage), end + n_words)
            
            cappedPassage = passage[left:right]
            marked_comp = marked_comp[left:right]
            
            
            X_train_comp_all.append(cappedPassage)
            X_train_comp_with_answer_marked_all.append(marked_comp)
            X_train_ans_all.append(answer)
            Y_train_ques_all.append(question)
        except Exception as e:
            invalid = invalid+1
print(invalid)

2429


In [8]:
c = list(zip(X_train_comp_all,X_train_ans_all, Y_train_ques_all,X_train_comp_with_answer_marked_all))
np.random.shuffle(c)
X_train_comp_all_shuffled,X_train_ans_all_shuffled, Y_train_ques_all_shuffled, X_train_comp_with_answer_marked_all_shuffled = zip(*c)


In [9]:
X_train_comp_with_answer_marked_all_shuffled[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.])

In [10]:
from collections import Counter
import operator
def findKMostFrequentWords(k):
    ctr = Counter([item for sublist in X_train_comp_all_shuffled for item in sublist] + [item for sublist in Y_train_ques_all_shuffled for item in sublist])
    sorted_ctr = sorted(ctr.items(), key=operator.itemgetter(1), reverse=True)
    return [item[0] for item in sorted_ctr[0:k]]

In [11]:
wordToTake = 200000
words = findKMostFrequentWords(100000)

In [12]:
_word_to_idx_reduced = {}
_idx_to_word_reduced = []


def _add_word_reduced(word):
    idx = len(_idx_to_word_reduced)
    _word_to_idx_reduced[word] = idx
    _idx_to_word_reduced.append(word)
    return idx


UNKNOWN_TOKEN = _add_word_reduced(UNKNOWN_WORD)
START_TOKEN = _add_word_reduced(START_WORD)
END_TOKEN = _add_word_reduced(END_WORD)




dimensions = glove.shape[1]
reduced_glove = []
reduced_glove.append(np.zeros(dimensions))
reduced_glove.append(-np.ones(dimensions))
reduced_glove.append(np.ones(dimensions))

for word in words:
    l = look_up_word(word)
    if(l != UNKNOWN_TOKEN):
        idx = _add_word_reduced(word)
        reduced_glove.append(glove[l])
        if(len(reduced_glove) == wordToTake):
            break
        
def look_up_word_reduced(word):
    return _word_to_idx_reduced.get(word, UNKNOWN_TOKEN)


def look_up_token_reduced(token):
    return _idx_to_word_reduced[token]

reduced_glove = np.array(reduced_glove)


In [13]:
common_words_set_tokens = set([look_up_word_reduced(i) for i in set(stopwords.words('english'))]) - {UNKNOWN_TOKEN, START_TOKEN}

In [14]:
len(common_words_set_tokens)

137

In [15]:
!hostname
print("Invalid = ",invalid)
print("Glove shape = ", reduced_glove.shape)

cuda2.cims.nyu.edu
Invalid =  2429
Glove shape =  (59528, 300)


In [92]:
examples_to_take_train = 500

X_train_comp = X_train_comp_all_shuffled[0:examples_to_take_train]
X_train_ans = X_train_ans_all_shuffled[0:examples_to_take_train]
Y_train_ques = Y_train_ques_all_shuffled[0:examples_to_take_train]
X_train_comp_with_answer_marked = X_train_comp_with_answer_marked_all_shuffled[0:examples_to_take_train]

In [93]:
max_document_len = len(max(X_train_comp,key=len))
max_answer_len = len(max(X_train_ans,key=len))
max_question_len = len(max(Y_train_ques,key=len)) + 1

In [94]:
document_tokens = np.full((examples_to_take_train, max_document_len), END_TOKEN,dtype=np.int32)
document_lengths = np.zeros(examples_to_take_train, dtype=np.int32)

answer_labels = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
answer_lengths = np.zeros(examples_to_take_train, dtype=np.int32)

question_input_tokens = np.full((examples_to_take_train, max_question_len), END_TOKEN, dtype=np.int32)
question_output_tokens = np.full((examples_to_take_train, max_question_len), END_TOKEN, dtype=np.int32)
question_lengths = np.zeros(examples_to_take_train, dtype=np.int32)


In [95]:
for i in range(examples_to_take_train):
    answer_labels[i, 0:len(X_train_comp_with_answer_marked[i])] = X_train_comp_with_answer_marked[i]
    for j, word in enumerate(X_train_comp[i]):
        document_tokens[i, j] = look_up_word_reduced(word)
    document_lengths[i] = len(X_train_comp[i])

    answer_lengths[i] = len(X_train_ans[i])
    
    question_input_words = ([START_WORD] + Y_train_ques[i])
    question_output_words = (Y_train_ques[i] + [END_WORD])

    for j, word in enumerate(question_input_words):
            question_input_tokens[i, j] = look_up_word_reduced(word)
    for j, word in enumerate(question_output_words):
        question_output_tokens[i, j] = look_up_word_reduced(word)
    question_lengths[i] = len(question_input_words)    

In [96]:
def create_vocabulary(data):
    flat_list = [item for sublist in data for item in sublist]
    vocabulary = sorted(set(flat_list))
    vocabulary.append("<UNK>")
    vocabulary.append("unk")
    vocabulary.append("eos")
    vocabulary = ["<EOS>"] + vocabulary
    word_to_index = { word:i for i,word in enumerate(vocabulary) }
    index_to_word = { i:word for i,word in enumerate(vocabulary) }
    return (vocabulary,word_to_index,index_to_word)

In [97]:
print(reduced_glove.shape)
vocabulary_comp,word_to_index_comp,index_to_word_comp = create_vocabulary(X_train_comp + Y_train_ques)
print(len(vocabulary_comp))
print(word_to_index_comp["?"])
print(word_to_index_comp["what"])

(59528, 300)
4397
282
4302


In [98]:
def create_one_hot_vector(data,vocabulary,word_to_index,index_to_word, maxLen):
    one_hot = np.zeros([maxLen,len(vocabulary)])
    for i,word in enumerate(data):
        if i >= maxLen:
            break
        if(word not in word_to_index):
            word = "<UNK>"
        one_hot[i][word_to_index[word]] = 1
    return one_hot

def create_one_hot_vector_from_indices(data,maxLen,vocabulary):
    one_hot = np.zeros([maxLen,len(vocabulary)])
    for i,indice in enumerate(data):
        if i >= maxLen:
            break
        one_hot[i][int(indice)] = 1
    return one_hot


def create_one_hot_training_Set(data,maxLen,vocabulary):
    one_hot_data = np.zeros([data.shape[0],maxLen,len(vocabulary)])
    for i in range(data.shape[0]):
        one_hot_data[i] = create_one_hot_vector_from_indices(data[i],maxLen,vocabulary)
    return one_hot_data




In [99]:
def context_to_indices_glove(X,max_len):
    
    m = len(X)                                 
    
    X_indices = np.full([m,max_len],look_up_word_reduced(END_WORD))
    
    for i in range(m):
        j = 0
        for w in X[i]:
            if(j>=max_len):
                break;
            
            X_indices[i, j] = look_up_word_reduced(w)
            j = j+1
    return X_indices

In [146]:
def generate_valid_tokens_vocab(X, Y):
    
    m = len(X)                                 
    
    X_indices = np.full([m,reduced_glove.shape[0]],0)
    lengths = np.zeros(m)
    
    for i in range(m):
        #s = set(X[i]).union(common_words_set_tokens) - {UNKNOWN_TOKEN}
        s = set(X[i]).union(set(Y[i])) - {UNKNOWN_TOKEN}
        for w in s:
            X_indices[i, w] = 1
        lengths[i] = len(s)
    return X_indices, lengths

In [147]:
document_tokens = context_to_indices_glove(X_train_comp, max_document_len)

In [148]:
valid_tokens, valid_tokens_length = generate_valid_tokens_vocab(document_tokens, question_output_tokens)

In [149]:
valid_tokens_length[100]

24.0

## Batch Data Preparation

In [155]:
max_question_len

29

In [156]:
import math

def createBatch(inputs,batch_size,shuffle=False):
    outputs = []
    start = 0
    while start < len(inputs[0]):
        end = min(len(inputs[0]), start + batch_size)
        output = {'document_tokens':[],
                    'document_lengths':[],
                    'answer_labels':[],
                    'answer_lengths': [],
                    'question_input_tokens':[],
                    'question_output_tokens':[],
                    'question_lengths':[],
                    'valid_tokens': [],
                    'valid_tokens_length': []}
        
        for index,inp in enumerate(inputs):
            #maxD = max(inputs[1][start:start+batch_size])
            maxD = max_document_len
            maxA = max(inputs[3][start:start+batch_size])
            maxQ = max_question_len
            
            if index == 0:
                output['document_tokens'].append(inp[start:end,0:maxD])
            elif index==1:
                output['document_lengths'].append(inp[start:end])
            elif index==2:
                output['answer_labels'].append(inp[start:end,0:maxD])
            elif index==3:
                output['answer_lengths'].append(inp[start:end])
            elif index==4:
                output['question_input_tokens'].append(inp[start:end, 0:maxQ])
            elif index==5:
                output['question_output_tokens'].append(inp[start:end, 0:maxQ])
            elif index==6:
                output['question_lengths'].append(inp[start:end])
            elif index==7:
                output['valid_tokens'].append(inp[start:end, 0:reduced_glove.shape[0]])
            elif index == 8:
                output['valid_tokens_length'].append(inp[start:end])
        
        output["document_tokens"] = np.array(output["document_tokens"])
        output["document_lengths"] = np.array(output["document_lengths"])
        output["answer_labels"] = np.array(output["answer_labels"])
        output["answer_lengths"] = np.array(output["answer_lengths"])
        output["question_input_tokens"] = np.array(output["question_input_tokens"])
        output["question_output_tokens"] = np.array(output["question_output_tokens"])
        output["question_lengths"] = np.array(output["question_lengths"])
        output["valid_tokens"] = np.array(output["valid_tokens"])
        output["valid_tokens_length"] = np.array(output["valid_tokens_length"])
        outputs.append(output)
        start = start + batch_size
            
    return outputs


In [157]:
batch_size = 32
batch_input = createBatch([document_tokens,document_lengths,answer_labels,answer_lengths,question_input_tokens,question_output_tokens,question_lengths,valid_tokens, valid_tokens_length]
                    ,batch_size)
for b in batch_input:
    for k, v in b.items():
        b[k] = v.squeeze()
number_of_batches = len(batch_input)
print("Number of batches = ", number_of_batches)

Number of batches =  16


In [158]:
#split = int(0.8 * number_of_batches)
#batch_input_test = batch_input[split:]
#batch_input = batch_input[0:split]

In [159]:
batch_input[0]['valid_tokens'].shape

(32, 59528)

In [160]:
len(batch_input)

16

## Model

In [161]:
USE_CUDA = torch.cuda.is_available()
#USE_CUDA = False
print('Use_CUDA={}'.format(USE_CUDA))
if USE_CUDA:
    # You can change device by `torch.cuda.set_device(device_id)`
    print('current_device={}'.format(torch.cuda.current_device()))

Use_CUDA=True
current_device=0


In [162]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [163]:
def batchwise_sample(gen, num_samples, batch_size, comp, documentEncoder, embedder):
    """
    Sample num_samples samples batch_size samples at a time from gen.
    Does not require gpu since gen.sample() takes care of that.
    """

    samples = []
    start = 0
    for i in range(int(math.ceil(num_samples/float(batch_size)))):
        batch_comp = comp[start:min(start+batch_size, num_samples)]
        samples.append(gen.sample(batch_comp.shape[0], batch_comp, documentEncoder, embedder).transpose(1,0))
        start+=batch_size

    return torch.cat(samples, START_TOKEN)[:num_samples].transpose(0,1)

In [164]:
def prepare_discriminator_data(pos_samples, neg_samples, comp = None, gpu=False):
    """
    Takes positive (target) samples, negative (generator) samples and prepares inp and target data for discriminator.
    Inputs: pos_samples, neg_samples
        - pos_samples: pos_size x seq_len
        - neg_samples: neg_size x seq_len
    Returns: inp, target
        - inp: (pos_size + neg_size) x seq_len
        - target: pos_size + neg_size (boolean 1/0)
    """

    inp = torch.cat((pos_samples, neg_samples), 0).type(torch.LongTensor)
    target = torch.ones(pos_samples.size()[0] + neg_samples.size()[0])
    target[pos_samples.size()[0]:] = 0
    

    # shuffle
    perm = torch.randperm(target.size()[0])
    target = target[perm]
    inp = inp[perm]

    if gpu:
        inp = inp.cuda()
        target = target.cuda()
    
    if comp is not None :
        comp = comp.repeat(2,1)
        if gpu:
            comp = comp[perm.cuda()]
        else:
            comp = comp[perm]
        return inp,target,comp
        

    return inp, target

In [165]:
def prepare_generator_batch(samples, start_letter=START_TOKEN, gpu=False):
    """
    Takes samples (a batch) and returns
    Inputs: samples, start_letter, cuda
        - samples: batch_size x seq_len (Tensor with a sample in each row)
    Returns: inp, target
        - inp: batch_size x seq_len (same as target, but with start_letter prepended)
        - target: batch_size x seq_len (Variable same as samples)
    """

    batch_size, seq_len = samples.size()

    inp = torch.zeros(batch_size, seq_len)
    target = samples
    inp[:, 0] = start_letter
    inp[:, 1:] = target[:, :seq_len-1]

    inp = Variable(inp).type(torch.LongTensor)
    target = Variable(target).type(torch.LongTensor)

    if gpu:
        inp = inp.cuda()
        target = target.cuda()

    return inp, target

### Document Encoder

In [166]:
class Embedder(nn.Module):
    def __init__(self, input_size, output_size):
        super(Embedder, self).__init__()
        self.embedding = nn.Embedding(input_size, output_size)
        
        # TODO: Verify
        self.embedding.weight = nn.Parameter(torch.from_numpy(reduced_glove).float())
        self.embedding.weight.requires_grad = False
    def forward(self, x):
        return self.embedding(x)

In [167]:
class DocumentEncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(DocumentEncoderRNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers=num_layers, batch_first= True, bidirectional=True)

    def forward(self, x, hidden):
        _, hidden = self.gru(x, hidden)
        hidden = hidden.view(1, hidden.shape[1], hidden.shape[2]*2*self.num_layers)
        return hidden
    
    def initHidden(self, num_samples):
        result = Variable(torch.zeros(2 * self.num_layers, num_samples, self.hidden_size)) #2 for BiDirectional
        if USE_CUDA:
            result = result.cuda()
        return result

In [168]:
class DocumentEncoderDiscRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(DocumentEncoderDiscRNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size,num_layers=num_layers, batch_first= True, bidirectional=True)

    def forward(self, x, hidden):
        _, hidden = self.gru(x, hidden)
        return hidden
    
    def initHidden(self, num_samples):
        result = Variable(torch.zeros(2 * self.num_layers, num_samples, self.hidden_size)) #2 for BiDirectional
        if USE_CUDA:
            result = result.cuda()
        return result

### Generator

In [169]:
class Generator(nn.Module):

    def __init__(self, embedding_dim, embeddings,  hidden_dim, vocab_size, max_seq_len, gpu=False, oracle_init=False):
        super(Generator, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.embeddings = embeddings
        self.max_seq_len = max_seq_len
        self.vocab_size = vocab_size
        self.gpu = gpu

        self.gru = nn.GRU(embedding_dim, hidden_dim)
        self.gru2out = nn.Linear(hidden_dim, vocab_size)

        # initialise oracle network with N(0,1)
        # otherwise variance of initialisation is very small => high NLL for data sampled from the same model
        if oracle_init:
            for p in self.parameters():
                init.normal(p, 0, 1)

    def init_hidden(self, comp, num_examples, documentEncoder, embedder):
        comp = embedder(comp).float()
        h = documentEncoder(comp, documentEncoder.initHidden(num_examples))
        #h = Variable(torch.zeros(1, batch_size, self.hidden_dim))
        if self.gpu:
            h = h.cuda()
        
        return h

    def forward(self, inp, hidden): #Make sure hidden comes from the DocumentEncoder...
        """
        Embeds input and applies GRU one token at a time (seq_len = 1)
        """
        # input dim                                             # batch_size
        emb = self.embeddings(inp)                              # batch_size x embedding_dim
        emb = emb.view(1, -1, self.embedding_dim).float()       # 1 x batch_size x embedding_dim
        
        out, hidden = self.gru(emb, hidden)                     # 1 x batch_size x hidden_dim (out)
        out = self.gru2out(out.view(-1, self.hidden_dim))       # batch_size x vocab_size
        out = F.log_softmax(out, dim=-1)
        return out, hidden

    def sample(self, num_samples, comp, documentEncoder, embedder, start_letter=START_TOKEN):
        """
        Samples the network and returns num_samples samples of length max_seq_len.
        Outputs: samples, hidden
            - samples: num_samples x max_seq_length (a sampled sequence in each row)
        """

        samples = torch.zeros(num_samples, self.max_seq_len).type(torch.LongTensor)

        h = self.init_hidden(comp, num_samples, documentEncoder, embedder)
        inp = torch.autograd.Variable(torch.LongTensor([start_letter]*num_samples))

        if self.gpu:
            samples = samples.cuda()
            inp = inp.cuda()

        for i in range(self.max_seq_len):
            out, h = self.forward(inp, h)               # out: num_samples x vocab_size
            out = torch.multinomial(torch.exp(out), 1)  # num_samples x 1 (sampling from each row)
            samples[:, i] = out.data

            inp = out.view(-1)

        return samples

    def batchNLLLoss(self, inp, target, comp, documentEncoder, embedder, num_samples = batch_size):
        """
        Returns the NLL Loss for predicting target sequence.
        Inputs: inp, target
            - inp: batch_size x seq_len
            - target: batch_size x seq_len
            inp should be target with <s> (start letter) prepended
        """

        loss_fn = nn.NLLLoss()
        batch_size, seq_len = inp.size()
        inp = inp.permute(1, 0)           # seq_len x batch_size
        target = target.permute(1, 0)     # seq_len x batch_size
        h = self.init_hidden(comp, num_samples, documentEncoder, embedder)

        loss = 0
        for i in range(seq_len):
            out, h = self.forward(inp[i], h)
            loss += loss_fn(out, target[i])

        return loss     # per batch

    def batchSuppressionLoss(self, inp,valid_toks,valid_tok_len, comp,documentEncoder, embedder, num_samples = batch_size):
        batch_size, seq_len = inp.size()
        inp = inp.permute(1, 0)           # seq_len x batch_size
        h = self.init_hidden(comp, num_samples, documentEncoder, embedder)

        loss = 0
        for i in range(seq_len):
            out, h = self.forward(inp[i], h)
            _, vocab_size_o = out.size()
            out = torch.exp(out)
            l1 = torch.sum(torch.mul(out, (1-valid_toks)).float(), dim = 1)
            l2 = torch.sum(torch.mul(out, valid_toks.float()), dim = 1)
            
            l1 = torch.div(l1, (vocab_size_o - valid_tok_len))
            l2 = torch.div(l2, valid_tok_len)
            loss += (torch.sum(l1) - torch.sum(l2))
        return loss
    
    def batchDegeneracyLoss(self, inp, comp,documentEncoder, embedder, num_samples = batch_size):
        batch_size, seq_len = inp.size()
        inp = inp.permute(1, 0)           # seq_len x batch_size
        h = self.init_hidden(comp, num_samples, documentEncoder, embedder)

        loss = 0
        for i in range(seq_len):
            out, h = self.forward(inp[i], h)
            out = torch.exp(out)
            out = out.unsqueeze(2)
            a1 = out.transpose(1,2)
            a2 = torch.log(out)
            loss += torch.bmm(a1, a2)
        loss = torch.sum(loss) / (batch_size * seq_len)
        return loss
        

    def batchPGLoss(self, inp, target, reward, comp, documentEncoder, embedder):
        """
        Returns a pseudo-loss that gives corresponding policy gradients (on calling .backward()).
        Inspired by the example in http://karpathy.github.io/2016/05/31/rl/
        Inputs: inp, target
            - inp: batch_size x seq_len
            - target: batch_size x seq_len
            - reward: batch_size (discriminator reward for each sentence, applied to each token of the corresponding
                      sentence)
            inp should be target with <s> (start letter) prepended
        """

        batch_size, seq_len = inp.size()
        inp = inp.permute(1, 0)          # seq_len x batch_size
        target = target.permute(1, 0)    # seq_len x batch_size
        h = self.init_hidden(comp, batch_size, documentEncoder, embedder)

        loss = 0
        for i in range(seq_len):
            out, h = self.forward(inp[i], h)
            # TODO: should h be detached from graph (.detach())?
            for j in range(batch_size):
                print(target.data[i][j], out[j][target.data[i][j]])
                loss += -out[j][target.data[i][j]]*reward[j]     # log(P(y_t|Y_1:Y_{t-1})) * Q

        return loss/batch_size

In [170]:
embedder = Embedder(input_size = reduced_glove.shape[0], output_size = reduced_glove.shape[1])

In [171]:
gen = Generator(embedding_dim = reduced_glove.shape[1], 
                embeddings = embedder, 
                hidden_dim = reduced_glove.shape[1], 
                vocab_size = reduced_glove.shape[0], 
                max_seq_len = max_question_len, 
                gpu=USE_CUDA)
'''
disc = Discriminator(embedding_dim = reduced_glove.shape[1], 
                     embeddings = embedder, 
                     hidden_dim = reduced_glove.shape[1], 
                     vocab_size = reduced_glove.shape[0], 
                     max_seq_len = max_question_len, 
                     gpu=USE_CUDA)
'''

documentEncoder = DocumentEncoderRNN(reduced_glove.shape[1], int(reduced_glove.shape[1] / 2))

#documentEncoderDisc = DocumentEncoderDiscRNN(reduced_glove.shape[1], reduced_glove.shape[1])



if USE_CUDA:
    gen= gen.cuda()
    #disc = disc.cuda()
    documentEncoder = documentEncoder.cuda()
    #documentEncoderDisc = documentEncoderDisc.cuda()


In [172]:
def train_generator_MLE(gen, gen_opt, batch_input, epochs):
    """
    Max Likelihood Pretraining for the generator
    """
    for epoch in range(epochs):
        print('epoch %d : ' % (epoch + 1), end='')
        sys.stdout.flush()
        total_loss = 0
        total_loss_2 = 0
        total_loss_3 = 0
        print(" ")
        for i in range(len(batch_input)):
            print('batch %d/%d : ' % (i + 1, len(batch_input)), end='\r')
            inp, target, comp, valid_toks, valid_tok_len = batch_input[i]['question_input_tokens'], batch_input[i]['question_output_tokens'], batch_input[i]['document_tokens'], batch_input[i]['valid_tokens'], batch_input[i]['valid_tokens_length']
            if USE_CUDA:
                inp = torch.from_numpy(inp).long().cuda()
                target = Variable(torch.from_numpy(target).long().cuda())
                comp = Variable(torch.from_numpy(comp).long().cuda())
                valid_toks = Variable(torch.from_numpy(valid_toks).float().cuda())
                valid_tok_len = Variable(torch.from_numpy(valid_tok_len).float().cuda())
            else:
                inp = torch.from_numpy(inp).long()
                target = Variable(torch.from_numpy(target).long())
                comp = Variable(torch.from_numpy(comp).long())
                valid_toks = Variable(torch.from_numpy(valid_toks).float())
                valid_tok_len = Variable(torch.from_numpy(valid_tok_len).float())
                
            gen_opt.zero_grad()
            
            loss3 = gen.batchSuppressionLoss(inp,valid_toks,valid_tok_len,comp,documentEncoder, embedder, inp.shape[0])

            loss2 = gen.batchDegeneracyLoss(inp, comp, documentEncoder, embedder, inp.shape[0])
            
            loss = gen.batchNLLLoss(inp,target,comp,documentEncoder, embedder, inp.shape[0])
            
            net_loss = loss2 + loss3
            net_loss.backward()
            gen_opt.step()

            total_loss += loss.data[0]
            total_loss_2+= loss2.data[0]
            total_loss_3 += loss3.data[0] 
            sys.stdout.flush()


        print(' Total Loss = %.4f, Degeneracy Loss = %.4f, Suppression Loss = %.4f' % (total_loss, total_loss_2, total_loss_3))

In [173]:
train_param_gen = []
train_param_disc = []

for model in [gen, documentEncoder]:
    train_param_gen += [p for p in model.parameters() if p.requires_grad]

#for model in [disc, documentEncoderDisc]:
#    train_param_disc += [p for p in model.parameters() if p.requires_grad]
    
gen_optimizer = optim.Adam(train_param_gen, lr=1e-2)
#dis_optimizer = optim.Adagrad(train_param_disc)




In [174]:
print("Training Generator")
train_generator_MLE(gen,gen_optimizer, batch_input, 200)


Training Generator
epoch 1 :  
 Total Loss = 3866.8150, Degeneracy Loss = -89.2424, Suppression Loss = -292.0138
epoch 2 :  
 Total Loss = 6583.1402, Degeneracy Loss = -10.3828, Suppression Loss = -565.3411
epoch 3 :  
 Total Loss = 7660.3583, Degeneracy Loss = -10.5928, Suppression Loss = -565.8740
epoch 4 :  
 Total Loss = 7875.7777, Degeneracy Loss = -10.6726, Suppression Loss = -565.9584
epoch 5 :  
 Total Loss = 7904.0829, Degeneracy Loss = -10.7712, Suppression Loss = -566.0116
epoch 6 :  
 Total Loss = 7887.4467, Degeneracy Loss = -10.7080, Suppression Loss = -566.0754
epoch 7 :  
 Total Loss = 7843.6722, Degeneracy Loss = -10.8187, Suppression Loss = -566.1232
epoch 8 :  
 Total Loss = 7828.6109, Degeneracy Loss = -10.7778, Suppression Loss = -566.1671
epoch 9 :  
 Total Loss = 7803.0675, Degeneracy Loss = -10.7172, Suppression Loss = -566.2000
epoch 10 :  
 Total Loss = 7743.2244, Degeneracy Loss = -10.6820, Suppression Loss = -566.2265
epoch 11 :  
 Total Loss = 7688.3817, De

KeyboardInterrupt: 

In [175]:
def printTokens(tokens):
    for token in tokens:
        print(look_up_token_reduced(token), sep=' ', end=' ')
    print(" ")

In [176]:
batch_num = 13
x = gen.sample(batch_input[batch_num]['document_tokens'].shape[0], torch.from_numpy(batch_input[batch_num]['document_tokens']).cuda(), documentEncoder, embedder)
#loss2 = gen.batchSuppressionLoss(inp,valid_toks,comp,documentEncoder, embedder, inp.shape[0]) 
#loss = gen.batchNLLLoss(inp,target,comp,documentEncoder, embedder, inp.shape[0])
for i in range(x.shape[0]):
    print("----------------------------------------------------------------------------------------------------")
    print("Comprehension : ")
    printTokens(batch_input[batch_num]['document_tokens'][i])
    print("*****************************************************************************************************")
    print("Ground Truth Question : ")
    printTokens(batch_input[batch_num]['question_input_tokens'][i])
    print("*****************************************************************************************************")
    print("Generated Question : ")
    printTokens(x[i])
    print("----------------------------------------------------------------------------------------------------")


----------------------------------------------------------------------------------------------------
Comprehension : 
ammunition . in addition , the prussian artillery batteries had 30 % more guns than their french counterparts . the prussian guns typically <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END>  
*****************************************************************************************************
Ground Truth Question : 
<START> the prussian artillery batteries had what percentage more of guns than the french ? <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END>  
*****************************************************************************************************
Generated Question : 
<END> <END> <END> <END> <END> ? ? <END> <END> ? <END> <END> <END> <END> <END> ? ? <END> ? ? ? <END> <END> <END> ? <END> <END> ? <END>  
-------------------------------------------------------

<START> when did toyotomi hideyoshi send an army to korea a second time ? <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END>  
*****************************************************************************************************
Generated Question : 
<END> <END> ? ? <END> ? <END> ? <END> ? <END> <END> ? ? <END> <END> <END> <END> ? ? <END> <END> ? <END> <END> <END> <END> <END> ?  
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Comprehension : 
teaching of what he called `` inert ideas '' – ideas that are disconnected scraps of information , with no application to real life or culture . he opined that `` education with inert ideas is <END> <END> <END> <END> <END> <END> <END>  
*****************************************************************************************************
Ground Truth Questio

In [ ]:
class Discriminator(nn.Module):

    def __init__(self, embedding_dim, embeddings,  hidden_dim, vocab_size, max_seq_len,  gpu=False, dropout=0.2):
        super(Discriminator, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.max_seq_len = max_seq_len
        self.gpu = gpu

        self.embeddings = embeddings
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=2, bidirectional=True, dropout=dropout)
        self.gru2hidden = nn.Linear(2*2*hidden_dim, hidden_dim)
        self.dropout_linear = nn.Dropout(p=dropout)
        self.hidden2out = nn.Linear(hidden_dim, 1)

    def init_hidden(self, comp, num_examples, documentEncoderDisc, embedder):
        comp = embedder(comp).float()
        h = documentEncoderDisc(comp, documentEncoderDisc.initHidden(num_examples))
        hidden = Variable(torch.zeros(2*2*1, num_examples, self.hidden_dim))
        if self.gpu:
            hidden = hidden.cuda()
        
        #hidden[0:2,:,:] = h

        return hidden
        

    def forward(self, input, hidden):
        # input dim                                                # batch_size x seq_len
        emb = self.embeddings(input)                               # batch_size x seq_len x embedding_dim
        emb = emb.permute(1, 0, 2).float()                                 # seq_len x batch_size x embedding_dim
        _, hidden = self.gru(emb, hidden)                          # 4 x batch_size x hidden_dim
        hidden = hidden.permute(1, 0, 2).contiguous()              # batch_size x 4 x hidden_dim
        out = self.gru2hidden(hidden.view(-1, 4*self.hidden_dim))  # batch_size x 4*hidden_dim
        out = F.tanh(out)
        out = self.dropout_linear(out)
        out = self.hidden2out(out)                                 # batch_size x 1
        out = F.sigmoid(out)
        return out

    def batchClassify(self, inp, num_examples,comp, documentEncoderDisc, embedder):
        """
        Classifies a batch of sequences.
        Inputs: inp
            - inp: batch_size x seq_len
        Returns: out
            - out: batch_size ([0,1] score)
        """

        h = self.init_hidden(comp, num_examples, documentEncoderDisc, embedder)
        out = self.forward(inp, h)
        return out.view(-1)

    def batchBCELoss(self, inp, target, comp, num_examples, documentEncoderDisc, embedder):
        """
        Returns Binary Cross Entropy Loss for discriminator.
         Inputs: inp, target
            - inp: batch_size x seq_len
            - target: batch_size (binary 1/0)
        """

        loss_fn = nn.BCELoss()
        h = self.init_hidden(comp, num_examples, documentEncoderDisc, embedder)
        out = self.forward(inp, h)
        return loss_fn(out, target)


In [ ]:
def train_discriminator(discriminator, dis_opt, train_param_disc, generator, d_steps, epochs):
    """
    Training the discriminator on real_data_samples (positive) and generated samples from generator (negative).
    Samples are drawn d_steps times, and the discriminator is trained for epochs epochs.
    """

    # generating a small validation set before training (using oracle and generator)
    n = 128
    l = random.sample(range(len(batch_input)), n//batch_size)
    pos_val = None
    neg_val = None
    for i in l:
        pos = Variable(torch.from_numpy(batch_input[i]['question_output_tokens']))
        if USE_CUDA:
            neg = gen.sample(batch_size, Variable(torch.from_numpy(batch_input[i]['document_tokens']).long()).cuda(), documentEncoder, embedder)
        else:
            neg = gen.sample(batch_size, Variable(torch.from_numpy(batch_input[i]['document_tokens']).long()), documentEncoder, embedder)
        if pos_val is None:
            pos_val = pos
            neg_val = neg
        else:
            pos_val = torch.cat((pos_val, pos), dim=0)
            neg_val = torch.cat((neg_val, neg), dim=0)

    if USE_CUDA:
        pos_val = pos_val.cuda()
        neg_val = neg_val.cuda()
        
    
    
    val_inp, val_target = prepare_discriminator_data(pos_val, Variable(neg_val).int())

    for d_step in range(d_steps):
        comp = None
        real_data = None
        for batch in batch_input:
            if comp is None:
                comp = batch['document_tokens']
                real_data = batch['question_output_tokens']
            else:
                comp = np.concatenate([comp,batch['document_tokens']], axis = 0)
                real_data = np.concatenate([real_data,batch['question_output_tokens']], axis = 0)
                
        comp = Variable(torch.from_numpy(comp).long())
        real_data = Variable(torch.from_numpy(real_data).long())
        
        if USE_CUDA:
            comp = comp.cuda()
            real_data=real_data.cuda()
        
        s = batchwise_sample(generator, examples_to_take_train, batch_size, comp, documentEncoder, embedder)
        dis_inp, dis_target, comp = prepare_discriminator_data(real_data, Variable(s), comp, gpu=USE_CUDA)
        for epoch in range(epochs):
            print('d-step %d epoch %d : ' % (d_step + 1, epoch + 1), end='')
            sys.stdout.flush()
            total_loss = 0
            total_acc = 0

            for i in range(0, dis_inp.shape[0], batch_size):
                inp, target, comp_batch = dis_inp[i:i + batch_size], dis_target[i:i + batch_size], comp[i:i + batch_size]
                dis_opt.zero_grad()
                #torch.nn.utils.clip_grad_norm(train_param_disc, 0.25)
                
                out = discriminator.batchClassify(inp, inp.shape[0], comp_batch, documentEncoderDisc, embedder)
                
                loss_fn = nn.BCELoss()

                loss = loss_fn(out, Variable(target))
                loss.backward()
                dis_opt.step()

                total_loss += loss.data[0]

                #total_acc += np.sum((out.data>0.5)==(target>0.5))

                if (i / batch_size) % math.ceil(math.ceil(2 * examples_to_take_train / float(
                        batch_size)) / 10.) == 0:  # roughly every 10% of an epoch
                    print('.', end='')
                    sys.stdout.flush()

            print("Total_loss : ", total_loss)
            #total_loss /= math.ceil(2 * examples_to_take_train / float(batch_size))
            #total_acc /= float(2 * examples_to_take_train)

            #val_pred = discriminator.batchClassify(val_inp)
            #print(' average_loss = %.4f, train_acc = %.4f, val_acc = %.4f' % (
            #    total_loss, 0, 0))
            #torch.sum((val_pred>0.5)==(val_target>0.5)).data[0]/200.

In [ ]:
def train_generator_PG(gen, gen_opt, dis, documentEncoder, embedder):
    """
    The generator is trained using policy gradients, using the reward from the discriminator.
    Training is done for num_batches batches.
    """
    
    total_loss = 0
    for batch in batch_input:
        comp = Variable(torch.from_numpy(batch['document_tokens']))
        if USE_CUDA:
            comp = comp.cuda()
            
        s = gen.sample(batch['document_tokens'].shape[0], comp, documentEncoder, embedder)
        inp, target = prepare_generator_batch(s, start_letter=START_TOKEN, gpu=USE_CUDA)
        rewards = disc.batchClassify(target, target.shape[0], comp, documentEncoderDisc, embedder)

        gen_opt.zero_grad()
        pg_loss = gen.batchPGLoss(inp, target, rewards, comp, documentEncoder, embedder)
        total_loss += pg_loss.data[0]
        pg_loss.backward()
        gen_opt.step()
    

    print('Total PG Loss : ',  total_loss)

In [ ]:
print("training Discriminator")
train_discriminator(disc, dis_optimizer, train_param_disc, gen, 2, 10)
for i in range(100):
    print("Training Generator")
    train_generator_PG(gen, gen_optimizer, disc, documentEncoder, embedder)
    print("training Discriminator")
    train_discriminator(disc, dis_optimizer, train_param_disc, gen, 2, 10)
    

In [ ]:
x = batchwise_sample(gen, 32, 32, torch.from_numpy(batch_input[0]['document_tokens']).cuda(), documentEncoder, embedder)

In [ ]:
x.shape

In [ ]:
for i in x:
    for j in i:
        print(look_up_token_reduced(j), end=' ')
    print(" ")

In [ ]:
comp = Variable(torch.from_numpy(batch_input[5]['document_tokens']))
if USE_CUDA:
    comp = comp.cuda()

In [ ]:
s = gen.sample(comp.shape[0], comp, documentEncoder, embedder)
inp, target = prepare_generator_batch(s, start_letter=START_TOKEN, gpu=USE_CUDA)

In [ ]:
rewards = disc.batchClassify(target, target.shape[0], comp, documentEncoderDisc, embedder)

In [ ]:
pg_loss = gen.batchPGLoss(inp, target, rewards, comp, documentEncoder, embedder)

In [ ]:
pg_loss

In [ ]:
rewards.shape

In [ ]:
batch_input[0]['valid_tokens_length']